In [1]:
import pandas as pd
import numpy as np
import os
import pdfplumber as pdfp
import tabula
import re
import PyPDF2

In [12]:
# diretório
diretorio = r'C:\Users\Computadores Gamer\OneDrive\Área de Trabalho\contas_ape'
diretorio = diretorio.replace('\\','/')
os.chdir(diretorio)

print(os.getcwd())

C:\Users\Computadores Gamer\OneDrive\Área de Trabalho\contas_ape


In [14]:
# arquivos
os.listdir()

['boleto_condominio_0823.pdf',
 'boleto_enel_0823.pdf',
 'boleto_quintoandar_0823.pdf']

In [ ]:
# boleto do condominio (valor e data de vencimento)
